In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from torch import nn
import torch.nn.functional as F

from torch.autograd import Variable
import torch.utils.data as Data
import torchvision

from torchvision import transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.patheffects as PathEffects
# from centerLoss import CenterLoss
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.cm as cm
%matplotlib inline

In [2]:
print(f"torch.cuda.is_available() = {torch.cuda.is_available()}")
print(f"torch.cuda.device_count() = {torch.cuda.device_count()}")
if 'CUDA_VISIBLE_DEVICES' in os.environ:
    print(f"os.environ['CUDA_VISIBLE_DEVICES'] = {os.environ['CUDA_VISIBLE_DEVICES']}")
else:
    print(f"没有设置环境变量 'CUDA_VISIBLE_DEVICES'")

torch.cuda.is_available() = True
torch.cuda.device_count() = 1
os.environ['CUDA_VISIBLE_DEVICES'] = 3


In [3]:
data_root = {
    "train":"train",
    "valid":"valid",
    "test":"test"
}

In [4]:
def train_tf(x):
   
    im_aug = transforms.Compose([
        transforms.Resize(120),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(96),
        transforms.ColorJitter(brightness=0.5, contrast=0.5, hue=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    x = im_aug(x)
    return x

def test_tf(x):
   
    im_aug = transforms.Compose([
        transforms.Resize(120),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    x = im_aug(x)
    return x

In [5]:
init_seed = 1
torch.manual_seed(init_seed)
torch.cuda.manual_seed(init_seed)
np.random.seed(init_seed) 

In [6]:
X1=ImageFolder(data_root["train"],transform=test_tf)  ## lazy loding
X2=ImageFolder(data_root["valid"],transform=test_tf)  ## lazy loding

X=torch.utils.data.ConcatDataset([X1,X2])

X_train,X_valid = torch.utils.data.random_split(X, [160000, 20000])
train_dataloader= DataLoader(X_train,batch_size=64,shuffle=True)
valid_dataloader= DataLoader(X_valid,batch_size=64,shuffle=True)

In [7]:
# X_train=ImageFolder(data_root["train"],transform=train_tf)  ## lazy loding
# train_dataloader= DataLoader(X_train,batch_size=64,shuffle=True)
# X_valid=ImageFolder(data_root["valid"],transform=test_tf)  ## lazy loding
# valid_dataloader= DataLoader(X_valid,batch_size=64,shuffle=True)

In [8]:
class CrossEntropyLabelSmooth(nn.Module):
    def __init__(self, num_classes, epsilon):
        super(CrossEntropyLabelSmooth, self).__init__()
        self.num_classes = num_classes
        self.epsilon = epsilon
        self.logsoftmax = nn.LogSoftmax(dim=1)
    def forward(self, inputs, targets):
        log_probs = self.logsoftmax(inputs)
        targets = torch.zeros_like(log_probs).scatter_(1, targets.unsqueeze(1), 1)
        targets = (1 - self.epsilon) * targets + self.epsilon / self.num_classes
        loss = (-targets * log_probs).mean(0).sum()
        return loss
    
class ConvNet(torch.nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.extract = torch.nn.Sequential(
            torchvision.models.resnet152(pretrained=True).conv1,
            torchvision.models.resnet152(pretrained=True).bn1,
            torchvision.models.resnet152(pretrained=True).relu,
            torchvision.models.resnet152(pretrained=True).maxpool,
            torchvision.models.resnet152(pretrained=True).layer1,
            torchvision.models.resnet152(pretrained=True).layer2,
            torchvision.models.resnet152(pretrained=True).layer3,
            torchvision.models.resnet152(pretrained=True).layer4,
            torchvision.models.resnet152(pretrained=True).avgpool,   
        )
        self.feat = torch.nn.Sequential(
            torch.nn.Linear(2048, 256),
        )
        for index,net in enumerate(self.feat):
            if index==0 :
                torch.nn.init.xavier_uniform_(net.weight)      
        self.pred = torch.nn.Sequential(
            torch.nn.Linear(256, 10),
        )  
        for index,net in enumerate(self.pred):
            if index==0:
                torch.nn.init.xavier_uniform_(net.weight)
                
    def forward(self, x):
        x = self.extract(x)    
        x = x.view(-1, 2048)
        feat = self.feat(x)
        pred = F.log_softmax(self.pred(feat),1)
        return feat ,pred

In [9]:
model = ConvNet()
checkpoint = torch.load("old_model/init1240.pkl")
model.load_state_dict(checkpoint['model'])

for param in model.parameters():
    param.requires_grad = False
for param in model.feat.parameters():
    param.requires_grad = True
for param in model.pred.parameters():
    param.requires_grad = True


model=model.cuda()

optimizer4nn = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), 
                               lr=1e-3, 
                               momentum=0.9, 
                               weight_decay=0.0005)    

nllloss = CrossEntropyLabelSmooth(10,0.1).cuda()

scheduler  = lr_scheduler.CosineAnnealingLR(optimizer4nn, T_max=5,eta_min=4e-08)


In [ ]:
from tqdm import tqdm_notebook
def train(train_loader, model, epoch):
    print("Training Epoch: {}".format(epoch))
    model.train()
    train_precision=0.0
    for step, (data, target) in enumerate(train_loader,1):
        data = Variable(data).cuda()
        target = Variable(target).cuda()
        feat, pred = model(data)
        _, pred_label = pred.max(dim=1)
        loss=nllloss(pred, target)
        optimizer4nn.zero_grad()
        loss.backward()
        optimizer4nn.step()
        train_precision += float(torch.sum(pred_label== target)) 
        if step % 350 == 0:
            print("Epoch: {} step: {} loss:{}".format(epoch, step,loss))
            
#     train_precision /= 90000.0
    train_precision /= 160000.0
    print("train accuracy: {}%".format(train_precision * 100))
    return train_precision
            
def test(test_loader, model, epoch):
    print("Predicting Epoch: {}".format(epoch))
    model.eval()
    precision=0.0
    for (data, target) in tqdm_notebook(test_loader):
        data = Variable(data).cuda()
        target = Variable(target).cuda()

        feature, pred = model(data)
        _, pred_label = pred.max(dim=1)
        
        precision += float(torch.sum(pred_label == target))

#     precision /=90000.0
    precision /=20000.0
    print("Validation accuracy: {}%".format(precision * 100))
    return precision
startepoch=0
best_precision=0.8
for epoch in range(startepoch,startepoch+40):
    train_precision=train(train_dataloader, model, epoch)
    precision=test(valid_dataloader, model, epoch)
    
    scheduler.step()
    if precision>best_precision:
        best_precision=precision
        state = {"model":model.state_dict(),"epoch":epoch,
                 "valid_acc":precision,
                 "train_acc":train_precision,
            }
        PATH ="old_model/init1240.pkl"
        torch.save(state, PATH)

Training Epoch: 0
Epoch: 0 step: 350 loss:0.5504523515701294
Epoch: 0 step: 700 loss:0.5234073400497437
Epoch: 0 step: 1050 loss:0.560851514339447
Epoch: 0 step: 1400 loss:0.5607202053070068
Epoch: 0 step: 1750 loss:0.552535355091095
Epoch: 0 step: 2100 loss:0.5318355560302734
Epoch: 0 step: 2450 loss:0.5588830709457397
train accuracy: 98.34875%
Predicting Epoch: 0



Validation accuracy: 90.72%
Training Epoch: 1
Epoch: 1 step: 350 loss:0.5406242609024048
Epoch: 1 step: 700 loss:0.5376715660095215
Epoch: 1 step: 1050 loss:0.5378513932228088
Epoch: 1 step: 1400 loss:0.5485162138938904
Epoch: 1 step: 1750 loss:0.542534351348877
Epoch: 1 step: 2100 loss:0.516972005367279
Epoch: 1 step: 2450 loss:0.5272921323776245
train accuracy: 98.43499999999999%
Predicting Epoch: 1



Validation accuracy: 90.85%
Training Epoch: 2
Epoch: 2 step: 350 loss:0.6113779544830322
Epoch: 2 step: 700 loss:0.562523365020752
Epoch: 2 step: 1050 loss:0.55600506067276
Epoch: 2 step: 1400 loss:0.5178607106208801
Epoch: 2 step: 1750 loss:0.5778690576553345
Epoch: 2 step: 2100 loss:0.5082237720489502
Epoch: 2 step: 2450 loss:0.570703387260437
train accuracy: 98.451875%
Predicting Epoch: 2



Validation accuracy: 90.735%
Training Epoch: 3
Epoch: 3 step: 350 loss:0.5102455615997314
Epoch: 3 step: 700 loss:0.5275911092758179


In [ ]:
exit()

In [ ]:
177.77
22.22
# state = {"model":model.state_dict(),"epoch":epoch,"valid_acc":precision }
# PATH ="old_model/cosin%d.pkl"% epoch

# torch.save(state, PATH)

In [18]:
# lossbox=[]
# accbox=[]

In [34]:
# startepoch=1
# for epochnum in range(startepoch,startepoch+10):
#     epoch_acc=0
#     for iternum,(X,Y) in enumerate(train_dataloader,1):
#         X,Y=X.cuda(),Y.cuda()
#         opt.zero_grad()
#         feat,Y_pred=model(X)
#         l=loss(Y_pred,Y)+ centerloss(target, feat)
        
#         l.backward()
#         opt.step()
        
#         if iternum%200==0:
#             Y_pred1=torch.argmax(Y_pred,1)
#             acc=float(torch.sum(Y_pred1==Y))/float(len(Y))
#             epoch_acc += 200.0*64*acc/90000.0
#             lossbox.append(l)
#             accbox.append(acc)
#             print("Iternum{:d}".format(iternum) + ", Minibatch Loss= " + "{:.6f}".format(l.data) +
#                     ", Training Accuracy= "+ "{:.5f}".format(acc))
        
#     Y_pred1=torch.argmax(Y_pred,1)
#     acc=float(torch.sum(Y_pred1==Y))/float(len(Y))
#     epoch_acc += 400.0*acc/90000.0
#     print("epochnum{:d}".format(epochnum) + ", Minibatch Loss= " + "{:.6f}".format(l.data)+
#           ", Training epoch_Accuracy= "+ "{:.5f}".format(epoch_acc))

In [ ]:
# class ConvNet(torch.nn.Module):
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         self.extract = torch.nn.Sequential(
#             torch.nn.Conv2d(1, 32, kernel_size=5, padding=2),
#             torch.nn.PReLU(),
#             torch.nn.Conv2d(32, 32, kernel_size=5, padding=2),
#             torch.nn.PReLU(),
#             torch.nn.MaxPool2d(2, 2),
#             torch.nn.Conv2d(32, 64, kernel_size=5, padding=2),
#             torch.nn.PReLU(),
#             torch.nn.Conv2d(64, 64, kernel_size=5, padding=2),
#             torch.nn.PReLU(),
#             torch.nn.MaxPool2d(2, 2),
#             torch.nn.Conv2d(64, 128, kernel_size=5, padding=2),
#             torch.nn.PReLU(),
#             torch.nn.Conv2d(128, 128, kernel_size=5, padding=2),
#             torch.nn.PReLU(),
#             torch.nn.MaxPool2d(2, 2),
#         )
#         self.feat = torch.nn.Linear(128*3*3, 2)
#         self.pred = torch.nn.Sequential(
#             torch.nn.Linear(2, 10)
#         )
 
#     def forward(self, x):
#         x = self.extract(x)
#         x = x.view(-1, 128*3*3)
#         feat = self.feat(x)
#         pred = F.log_softmax(self.pred(feat))
#         return feat, pred

In [10]:
X_test=ImageFolder(data_root["test"],transform=test_tf)  ## lazy loding
test_dataloader= DataLoader(X_test,batch_size=64,shuffle=True)

In [11]:
Y_predlist=[]
Y_test=[]
from tqdm import tqdm_notebook
for (X,Y) in tqdm_notebook(test_dataloader):
#     print(iternum)
    model.eval()
    X,Y=X.cuda(),Y.cuda()
    model=model.cuda()
    _,Y_pred=model(X)
#     print(Y_pred)
    Y_pred1=torch.argmax(Y_pred,axis=1)
    Y_predlist.extend(Y_pred1)
    Y_test.extend(Y)
Y_test=torch.Tensor(Y_test)
Y_predlist=torch.Tensor(Y_predlist)

In [12]:
acc=float(torch.sum(Y_predlist==Y_test))/float(len(Y_test))
acc

0.9100555555555555